# Textonic API
Če imaš težave z računanjem vektorjev; Funkcije za vektorizacijo z Textonic API.   
Spet kličem vektorizacijo na stolpcu 'Text' in dobim stolpec z vektorjem 'Embedding'

In [69]:
!pip install pandas numpy networkx numpy scikit-learn

In [70]:
import os
import pandas as pd

excel_path = os.path.join('..', 'data', 'Reworkan data 4.xlsx')
data = pd.read_excel(excel_path)
    
data.head()

,Article ID,Datum,Dan,Teden,Article created,Tip medija,Podtip medija,Media uuid,Medij,Rubric uuid,...,Pregled objav - citiranost virov,Razredi Indeksa medijske podobe,Doseg,Naklonjenost teme,Osrednja tema,Ključna sporočila,Fotografija,Omemba v naslovu,Omemba v podnaslovu,Tekst
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Thursday,14,2023-04-06 01:31:33.498,tisk,tisk,2fd717ed-78ba-4f63-b257-cd096acb6bda,Delo,bc228842-98cc-48ea-8be9-0107becb707a,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"V spomin Prof. dr. Marija Pečan, dr. med. (193..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Friday,14,2023-04-07 13:47:41.072,internet,splet,754da261-9aee-4a1a-b9d8-734cd409fabf,Zurnal24.si,e059ba7d-1a91-48f6-ac1c-d7ed703375ba,...,Objava z obema tipoma citatov,"2. razred (34,5 - 59 točk)",odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"Člani Strateškega sveta za prehrano, ki je med..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Saturday,15,2023-04-15 21:44:58.961,internet,splet,bc20546f-3a11-4061-90c2-2769468cd542,Delo.si,f57a09ac-450d-4df4-bc2b-a2c7f96a4322,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Člani komisije se niso opredeljevali do znakov...
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Tuesday,14,2023-04-04 08:22:00.122,tisk,tisk,a67b08ee-a757-40ff-8d7e-ab8fc8246eec,Jana,4dd54f38-07e1-448f-b96d-dad85e669834,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),zelo dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Monday,16,2023-04-17 19:40:59.639,internet,splet,754da261-9aee-4a1a-b9d8-734cd409fabf,Zurnal24.si,e059ba7d-1a91-48f6-ac1c-d7ed703375ba,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"""Tudi vsi v zdravstvenem sistemu bodo dobili s..."


V realnosti pazi na okoljske spremenljivke

In [71]:
print(f'TTNX_API_KEY is set: {len(os.getenv("TTNX_API_KEY"))}')

TTNX_API_KEY is set: 36


Definiram Textonic API funkcije za vektorizacijo:

In [72]:
import uuid
import numpy as np
from ttnx.api import call_textonic


def ttnx_embed(df: pd.DataFrame, col_name: str = 'Tekst', target_col_name: str = 'Embedding'):
    # Split the DataFrame into batches of 200 rows
    batch_size = 200
    num_batches = (len(df) + batch_size - 1) // batch_size
    id_column = 'Article ID'
    
    selected_columns = [id_column, col_name]
    responses = pd.DataFrame()
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        batch = df.iloc[start_idx:end_idx]
        request = {
            'requestId': str(uuid.uuid4()),
            'process': {
                'analysis': {
                    'steps': [
                        {
                            'step': 'doc_embed',
                            'engine': 'e5',
                            'model': 'multilingual-e5-base',
                            'attributes': [
                                {'named_sentence_filters': 'kl_transcript'}
                            ]
                        }
                    ]
                }
            },
            'documents': []
        }
        
        already_added_ids = set() 
        # Iterate over DataFrame rows and use selected columns and neglect duplicates
        for index, row in batch.iterrows():
            a = row[selected_columns].to_dict()
            if id_column not in a or not a[id_column]:
                continue
            if a[id_column] in already_added_ids:
                continue
            already_added_ids.add(a[id_column])
            if col_name not in a:
                continue
            if isinstance(a[col_name], float):
                body_text = str(a[col_name])
            else:
                body_text = a[col_name].strip()
            if not body_text:
                continue
            
            document = {
                'id': a[id_column],
                'title': a[id_column],
                'language': 'sl',
                'sections': [
                    {
                        'outline': 'body',
                        'data': body_text
                    }
                ]
            }
            if a[id_column] == '974bfe2d-d56b-11ed-9246-2b5ebef623ad':
                print(f"[{document}]")
                continue
            request['documents'].append(document)
        if not request['documents']:
            continue
        resp_obj = call_textonic('/api/public/ml/process', request)
        
        result = {
            id_column: [],
            target_col_name: []
        }
        
        for res_item in resp_obj['data']:
            for res in res_item['result']:
                if 'c' in res and 'v' in res and 'doc_embed' in res['c']:
                    vect = np.array(res['v'])
                    result[id_column].append(res_item['id'])
                    result[target_col_name].append(vect)
                    
        print(f'Requested/received/valid vector : {len(batch)}/{len(resp_obj["data"])}/{len(result[target_col_name])}')
        # Convert the collected responses to a DataFrame
        response_df = pd.DataFrame(result)
        responses = pd.concat([responses, response_df])
    
    orig_len = len(df)
    df = pd.merge(df, responses, on=id_column, how='left').dropna(subset=[target_col_name])
    print(f'Requested/Successful {orig_len}/{len(df)}')
    return df

Remote call to compute Textonic embeddings za 200 člankov na enkrat:

In [73]:
data = ttnx_embed(data, 'Tekst', 'Embedding')
data[['Article ID', 'Datum', 'Medij', 'Tekst', 'Embedding']].head(200)

Requested/received/valid vector : 200/177/177
[{'id': '974bfe2d-d56b-11ed-9246-2b5ebef623ad', 'title': '974bfe2d-d56b-11ed-9246-2b5ebef623ad', 'language': 'sl', 'sections': [{'outline': 'body', 'data': 'Žanr: Regionalne kronike  Prikaži manj'}]}]
Requested/received/valid vector : 200/179/179
Requested/received/valid vector : 200/185/185
Requested/received/valid vector : 200/185/185
Requested/received/valid vector : 200/186/186
Requested/received/valid vector : 200/179/179
Requested/received/valid vector : 200/178/178
Requested/received/valid vector : 200/189/189
Requested/received/valid vector : 200/188/188
Requested/received/valid vector : 200/190/190
Requested/received/valid vector : 200/183/183
Requested/received/valid vector : 200/182/182


Textonic error [process_error][[{'name': 'doc', 'value': '17710fb5-8328-11ee-b653-2fe3bf8a9fb0'}, {'name': 'engine', 'value': 'stanza'}, {'name': 'trace', 'value': "java.util.concurrent.ExecutionException: java.lang.Exception: Unable to invoke interpreter!\n\tat java.base@17.0.8/java.util.concurrent.FutureTask.report(FutureTask.java:122)\n\tat java.base@17.0.8/java.util.concurrent.FutureTask.get(FutureTask.java:191)\n\tat com.dropchop.textonic.processors.fasttext.FasttextLidProcessor.detectLanguage(FasttextLidProcessor.java:82)\n\tat com.dropchop.textonic.processors.fasttext.FasttextLidProcessor.process(FasttextLidProcessor.java:158)\n\tat com.dropchop.textonic.processors.fasttext.FasttextLidProcessor_Subclass.process$$superforward(Unknown Source)\n\tat com.dropchop.textonic.processors.fasttext.FasttextLidProcessor_Subclass$$function$$5.apply(Unknown Source)\n\tat io.quarkus.arc.impl.AroundInvokeInvocationContext.proceed(AroundInvokeInvocationContext.java:73)\n\tat io.quarkus.arc.impl.

TypeError: list indices must be integers or slices, not str

Funkcija za clusterizacijo

In [ ]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

def cluster_louvain(df: pd.DataFrame, col_name: str = 'Embedding', similarity_threshold: float = 0.96):
    print("Clustering ...")
    embeddings = np.array(df[col_name].to_list())
    labels = [0] * len(embeddings)
    x = cosine_similarity(embeddings, embeddings)
    similarity_matrix = x > similarity_threshold
    graph = nx.from_numpy_array(similarity_matrix)
    communities = nx.algorithms.community.louvain_communities(graph, resolution=0.1)
    for community in communities:
        initial_member = min(community)
        for member in community:
            labels[member] = initial_member

    df['Cluster'] = pd.Series(labels)
    print("Clustered")
    return df

Zračunam clustering

In [ ]:
data = cluster_louvain(data, 'Embedding', 0.96)  # <- similarity_threshold
data = data.drop(columns=['Embedding'])  # remove the vectors from data we don't need them anymore
data[['Article ID', 'Datum', 'Medij', 'Cluster', 'Tekst']].head(100)